In [10]:
import cv2
cv2.__version__

'4.5.1'

In [11]:
#program deteksi objek

import matplotlib.pyplot as plt
import cv2
import numpy as np

box=[]

# Load Yolo
net = cv2.dnn.readNet("yolov4-obj_best.weights","yolov4-obj.cfg")
#net = cv2.dnn.readNet("20_yolov3.weights","20_yolov3.cfg")
classes = []
with open("obj.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

color_blue = (255,0,0)
color_red = (0,0,255)
#colors = np.random.uniform(0, 255, size=(len(classes), 3))

# Loading image
image = cv2.imread("sampelGambar/SUNNY/camera8.jpg") 
#img = cv2.resize(imges, None, fx=1, fy=1)
height, width, channels = image.shape

# Detecting objects
#blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
blob = cv2.dnn.blobFromImage(image, 0.00392, (608, 608), (0, 0, 0), True, crop=False)
net.setInput(blob)
outs = net.forward(output_layers)

# Showing informations on the screen
class_ids = []
confidences = []
boxes = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.45:
            if class_id == 1:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.45, 0.4)

#print(indexes)
font = cv2.FONT_HERSHEY_PLAIN
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        
        box.append(boxes[i])
        #print(boxes[i])
        label = str(classes[class_ids[i]])
        #color = colors[i]
        cv2.rectangle(image, (x, y), (x + w, y + h), color_blue, 2)
        cv2.putText(image, label, (x, y + 30), font, 2, color_blue, 2)
        cv2.putText(image, str("{:.2f}".format(confidences[i])), (x, y + 50), font, 2, color_red, 2)
        
cv2.imshow("output1",image)
cv2.waitKey(0)
cv2.destroyAllWindows()
#cv2.imwrite("output1.jpg", image)


In [12]:
#Load koodinat npy
ROI_slot = np.load('coordinate/camera8.npy',allow_pickle=True)

In [13]:
#inisialisasi array
diagonal_slot = []                      
centroid_slot_x = []
centroid_slot_y = []
for i in range (len(ROI_slot)):            
    new = []
    new1 = []
    new2 = []
    new3 = [0,0]
    for j in (ROI_slot[i]):
        new.append(0)
        new1.append(0)
        new2.append(new3)
    diagonal_slot.append(new1)    
    centroid_slot_x.append(new)
    centroid_slot_y.append(new1)


area_slot = []                      
for i in range (len(ROI_slot)):            
    new = []
    for j in (ROI_slot[i]):
        new.append(0)
    area_slot.append(new)

    
for row in range(len(ROI_slot)):
    for slots in range(len(ROI_slot[row])):
        x1 = ROI_slot[row][slots][0][0]
        y1 = ROI_slot[row][slots][0][1]
        x2 = ROI_slot[row][slots][1][0]
        y2 = ROI_slot[row][slots][1][1]
        x3 = ROI_slot[row][slots][2][0]
        y3 = ROI_slot[row][slots][2][1]
        x4 = ROI_slot[row][slots][3][0]
        y4 = ROI_slot[row][slots][3][1]
        
        centroid_slot_x[row][slots] = (x3+x1)/2
        centroid_slot_y[row][slots] = (y3+y1)/2
        
        
        area_slot[row][slots] = (abs(((x1*y2-y1*x2) + (x2*y3-y2*x3) + (x3*y4-y3*x4) + (x4*y1-y4*x1))/2))

In [14]:
#import numpy as np

#arr = np.array(area_slot)
#print(arr)

In [15]:
#Export ke npy
#from numpy import save

#save("areaCNR/area8.npy",arr)

In [16]:
boxess = []
for i in range(len(indexes)):
    boxess.append([])
    boxess[i].append(box[i][0])
    boxess[i].append(box[i][1])
    boxess[i].append(box[i][2]+box[i][0])
    boxess[i].append(box[i][3]+box[i][1])

In [17]:
ROI_car = []
for i in range(len(indexes)):
    ROI_car.append([(boxess[i][0],boxess[i][3]), (boxess[i][0],boxess[i][1]), (boxess[i][2],boxess[i][1]), (boxess[i][2],boxess[i][3])])    


In [18]:
#Load images
image = cv2.imread("sampelGambar/SUNNY/camera8.jpg")
image = cv2.resize(image, (image.shape[1],image.shape[0]), interpolation = cv2.INTER_AREA)
plottedImage = image.copy()

In [19]:
#Menghitung RoI

from shapely.geometry import Polygon

IoU = []
Status = []
for i in range (len(ROI_slot)):            
    new = []
    new1 = []
    for j in (ROI_slot[i]):
        new.append(0)
        new1.append(0)
    IoU.append(new)
    Status.append(new1)

intersect=[]
count=0

for row in range(len(ROI_slot)):
    for slots in range(len(ROI_slot[row])):
        intersect=[]
        for car in range(len(ROI_car)):
            p = Polygon(ROI_slot[row][slots])
            q = Polygon(ROI_car[car])
            intersect.append(q.intersection(p).area)
        
        max_intrsct=max(intersect)   
        IoU[row][slots] = ((max_intrsct/area_slot[row][slots])*100)
        if IoU[row][slots] > 50:
            Status[row][slots] = 'OCCUPIED'
        else:
            Status[row][slots] = 'VACANT'
        #draw ROI_slot
        mod = cv2.polylines(plottedImage, np.int32([ROI_slot[row][slots]]), True, (255,0,0), thickness=2)
        #draw Status_slot
        mod = cv2.putText(plottedImage, Status[row][slots], (int(centroid_slot_x[row][slots])-10, int(centroid_slot_y[row][slots])),
                          cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 0, 255), 1, cv2.LINE_AA)

print(IoU)

[[27.9222481453325, 38.536610165331744, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6256994859706625, 77.06109061425275, 8.061631661044002, 0.0], [0.0, 7.3410781815192365, 90.45701915828643, 2.3269818545409104, 0.0, 82.3793029692603, 9.739261507132372, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.021485446845522057], [0.0, 0.0, 0.0, 5.68638261668982, 83.01384038008727, 77.08509191521291, 86.40678537369125, 77.38855112514804, 79.85489949888506, 74.37045810116352, 88.43136123503922, 82.03300778236428], [84.3707896651552, 92.07412349389328, 89.61775171502138, 98.72616152939602, 96.95055362087852, 85.91428982000203, 86.48169950437095], [0.0, 35.5618285893515, 35.98848929108484]]


In [20]:
for car in range(len(ROI_car)):
    mod = cv2.polylines(plottedImage, np.int32([ROI_car[car]]), True, (0,255,0), thickness=2)

In [24]:
cv2.imshow('Shapes', mod)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [33]:
#MASIH ERROR
for row in range(len(ROI_slot)):
    for slots in range(len(ROI_slot[row])):
        if Status[row][slots] == 'VACANT':
            continue
        else:
            for car in ROI_car:
                mod = cv2.bitwise_and(ROI_slot[row][slots],car)

TypeError: Expected Ptr<cv::UMat> for argument 'src1'

In [37]:
#MASIH ERROR
ROI_slot[0][6]

[[662, 68], [676, 95], [625, 101], [614, 65]]

In [ ]:
#cv2.imwrite('27April/Hasil_ED/SUNNY/camera1.jpg',mod)